In [85]:
import numpy as np
import astropy.io.fits
import random
import os
import os.path
from collections import Counter
from PIL import Image
from sklearn.cluster import KMeans
from astropy.time import Time
import csv
from statistics import stdev
from statistics import mean
import math
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
import cudf
from cuml.cluster import DBSCAN
from cuml.cluster import KMeans as cudkmean
import timeit
import collections


# Normalizing between [1,-1]
def normal(a):
    
    aa = a - a.min()
    bb = a.max() - a.min()
    cc = (2 * aa) / bb
    dd = cc - 1
    return dd


# Unnormal

def unnormal(norm, pure):
    ee = norm + 1
    ff = ee/2
    gg = ff * (pure.max() - pure.min())
    hh = gg + pure.min()
    return hh


# Getting Features

def getFeatures(Name):
    fileData = astropy.io.fits.open(Name)
    B = fileData[1].data
    B = np.array(B, dtype=np.float32)
    Binc = fileData[2].data
    Binc = np.array(Binc, dtype=np.float32)
    Bazi = fileData[3].data
    Bazi = np.array(Bazi, dtype=np.float32)
    DopF = fileData[4].data
    DopF = np.array(DopF, dtype=np.float32)
    #DopS = fileData[5].data
    #DopS = np.array(DopS, dtype=np.float64)
    SLFF = fileData[12].data
    SLFF = np.array(SLFF, dtype=np.float32)
    CI = fileData[32].data
    CI = np.array(CI, dtype=np.float32)


    return B, Binc, Bazi, DopF, SLFF, CI


# Removing that effect due to doppler shifts

def canNoise(Doppler):
    
    y = np.mean(Doppler, 0)
    x = np.array(range(Doppler.shape[1]))
    m, b = np.polyfit(x, y, 1)
    v = m*x + b
    out = Doppler - v

    return out

# Normalizing angles

def normangle(feature):
    
    
    a = np.cos(np.deg2rad(feature))
    
    return a
      

# Making a list for dimensions of observations
def dimlist(lists):
    
    dimList = list()


    for i in lists:

        file = astropy.io.fits.open(i)
        x, y = file[1].data.shape
        dimList.append((x, y))
    
    return dimList


# Concatenating all the observations

def concatfeatures(NewList):

    Btot, Binctot, DopFtot, SLFFtot, CItot = np.array([]), np.array([]),np.array([]), np.array([]),np.array([])



    for j in NewList:

        B, Binc, Bazi, DopF, SLFF, CI = getFeatures(j)
        Btot = np.concatenate((Btot, B), axis=None)
        Binctot = np.concatenate((Binctot, Binc), axis=None)
        DopFtot = np.concatenate((DopFtot, canNoise(DopF)), axis=None)
        SLFFtot = np.concatenate((SLFFtot, SLFF), axis=None)
        CItot = np.concatenate((CItot, CI), axis=None)
    
    
    return Btot, Binctot, DopFtot, SLFFtot, CItot

# Plot mappings
def pltmaps(labels, dimList, k):
    
    start = 0
    end = 0

    for s, tple in enumerate(dimList):

        end += tple[0] * tple[1]

        pltimg(labels[start:end], tple, s)

        start = end 
        
def pltimg(labels, tple, s):
    
    fileData = (NewList[s])
    v = fileData[42:-5]
    labless = labels.reshape(tple)
    plt.figure(figsize=(16,9))
    plt.imshow(labless, cmap="plasma", origin="bottom")
    plt.colorbar()
    plt.savefig(dir+'/'+ v + '.png')
    plt.close()

    
    

    
    
NorthDir='/glade/work/egeland/hinode_synoptic/north'
allfiles = [os.path.join(NorthDir, f) for f in sorted(os.listdir(NorthDir))]

# Removing all sav files
NewList=[item for i,item in enumerate(allfiles) if i%2==0]
b = []

for item in NewList:
    
    fileData = astropy.io.fits.open(item)
    a = fileData[0].header['YSCALE']
    if a > 0.16 :
        b.append(item)

NewList = []       
NewList = b
dimList = dimlist(NewList)

In [86]:
Btot, Binctot, DopFtot, SLFFtot, CItot = concatfeatures(NewList)


for it, item in enumerate(Btot):
    
    if item < 200:
        
        Btot[it] = 0
        Binctot[it] = 90
        SLFFtot[it] = 0
        
    elif item > 2000:
        
        Btot[it] = 2000
        

Btotn = normal(Btot)
Binctotn = normangle(Binctot)
DopFtotn = normal(DopFtot)
SLFFtotn = normal(SLFFtot)
CItotn = normal(CItot)

da = np.array([Btotn, Binctotn, DopFtotn, SLFFtotn, CItotn])
fda = np.transpose(da)

In [87]:
print(fda.shape)
cudf.set_allocator("managed")
X_df = pd.DataFrame({'fea%d'%i: fda[:, i] for i in range(fda.shape[1])})
X_gpu = cudf.DataFrame.from_pandas(X_df)

(22620160, 5)


In [94]:
start = timeit.default_timer()
k=20

cud = cudkmean(n_clusters = k)
cud.fit(X_gpu)
inert = cud.score(X_gpu)

In [95]:
cn = cud.cluster_centers_.as_matrix()
labels = cud.labels_.to_array()


In [96]:
dir = "cluster"+ str(k)
os.mkdir(dir)

cn[cn < -1] = -1
cn[cn > 1] = 1

clcode = np.array(range(k))
Bn = unnormal(cn[:, 0], Btot)
Bincn = np.degrees(np.arccos(cn[:, 1]))
DopFn = unnormal(cn[:, 2], DopFtot)
SLFFn = unnormal(cn[:, 3], SLFFtot)
CIn = unnormal(cn[:, 4], CItot)

cntrs = np.transpose(np.array([clcode, Bn, Bincn, DopFn, SLFFn, CIn]))





vv = cntrs[cntrs[:,5].argsort()]
neworder = np.array(range(1, k+1)).reshape(k,1)


cntrs = np.append(neworder, vv, axis = 1)




nwordr = vv[:,0]

gg = 1000
for ff in nwordr:
    labels[labels==ff] = gg
    gg += 1000

labels = labels/1000


count = collections.Counter(labels)
print(count)
mmbr = np.array([])

for i in range(1,k+1):
    mmbr = np.append(mmbr, count[i])
    
    
mmbr = mmbr.reshape(k, 1)
print(mmbr)
cntrs = np.append(mmbr, cntrs,axis = 1)


fields = ['membership', 'neworder', 'clcode', 'B', 'Binc', 'DopF', 'SLFF', 'CI']  

# data rows of csv file  
rows =  cntrs

# name of csv file  
filename = dir + "/"+ "cluterTable.csv"

with open(filename, 'w') as csvfile:  
            # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  

            # writing the fields  
    csvwriter.writerow(fields)  

            # writing the data rows  
    csvwriter.writerows(rows)
            
            
            
print('inertia: ', inert)
labels[labels!=1]=0

count = collections.Counter(labels)
print(count)

pltmaps(labels, dimList, k)

Counter({20.0: 10556891, 9.0: 8911024, 18.0: 400071, 7.0: 381603, 4.0: 307524, 14.0: 298935, 5.0: 267478, 17.0: 247246, 6.0: 229632, 3.0: 207075, 13.0: 184400, 16.0: 145760, 10.0: 115390, 8.0: 101117, 12.0: 92661, 11.0: 62904, 15.0: 56491, 19.0: 34574, 2.0: 16431, 1.0: 2953})
[[2.9530000e+03]
 [1.6431000e+04]
 [2.0707500e+05]
 [3.0752400e+05]
 [2.6747800e+05]
 [2.2963200e+05]
 [3.8160300e+05]
 [1.0111700e+05]
 [8.9110240e+06]
 [1.1539000e+05]
 [6.2904000e+04]
 [9.2661000e+04]
 [1.8440000e+05]
 [2.9893500e+05]
 [5.6491000e+04]
 [1.4576000e+05]
 [2.4724600e+05]
 [4.0007100e+05]
 [3.4574000e+04]
 [1.0556891e+07]]
inertia:  -466168.7784719217
Counter({0.0: 22617207, 1.0: 2953})


In [46]:
1!=2

True

In [103]:
import timeit

start = timeit.default_timer()

k= 35
gak_km = KMeans(n_clusters=k, n_jobs=-1).fit(fda)

stop = timeit.default_timer()

print('Time: ', stop - start) 

/glade/u/home/mnourela/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/cluster/_kmeans.py:974: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


Time:  4838.5732152380515


In [104]:
dir = "cluster"+ str(k)+"km"
os.mkdir(dir)

labels = gak_km.labels_
centers = gak_km.cluster_centers_
cn = np.transpose(centers)

cn[cn < -1] = -1
cn[cn > 1] = 1

clcode = np.array(range(k))
Bn = unnormal(cn[0], Btot)
Bincn = np.degrees(np.arccos(cn[1]))
DopFn = unnormal(cn[2], DopFtot)
SLFFn = unnormal(cn[3], SLFFtot)
CIn = unnormal(cn[4], CItot)

cntrs = np.transpose(np.array([clcode, Bn, Bincn, DopFn, SLFFn, CIn]))





vv = cntrs[cntrs[:,5].argsort()]
neworder = np.array(range(1, k+1)).reshape(k,1)


cntrs = np.append(neworder, vv, axis = 1)




nwordr = vv[:,0]

gg = 1000
for ff in nwordr:
    labels[labels==ff] = gg
    gg += 1000

labels = labels/1000


count = collections.Counter(labels)
print(count)
mmbr = np.array([])

for i in range(1,k+1):
    mmbr = np.append(mmbr, count[i])
    
    
print(mmbr)
mmbr = mmbr.reshape(k, 1)
cntrs = np.append(mmbr, cntrs,axis = 1)


fields = ['membership', 'neworder', 'clcode', 'B', 'Binc', 'DopF', 'SLFF', 'CI']  

# data rows of csv file  
rows =  cntrs

# name of csv file  
filename = dir + "/"+ "cluterTable.csv"

with open(filename, 'w') as csvfile:  
            # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  

            # writing the fields  
    csvwriter.writerow(fields)  

            # writing the data rows  
    csvwriter.writerows(rows)
            
            
            
print('inertia: ', gak_km.inertia_)

pltmaps(labels, dimList, k)

Counter({23.0: 2781388, 19.0: 2009922, 3.0: 1921775, 33.0: 1881153, 31.0: 1823798, 34.0: 1523354, 8.0: 1476079, 2.0: 1442444, 32.0: 1260515, 29.0: 900727, 21.0: 896523, 5.0: 852919, 35.0: 699751, 16.0: 282749, 11.0: 270105, 30.0: 263719, 13.0: 227869, 24.0: 227237, 26.0: 214485, 4.0: 210424, 10.0: 203044, 6.0: 175925, 9.0: 152458, 14.0: 142655, 27.0: 136352, 25.0: 131312, 15.0: 114186, 12.0: 93938, 20.0: 81208, 17.0: 65628, 22.0: 53468, 18.0: 51429, 28.0: 32819, 7.0: 15979, 1.0: 2823})
[   2823. 1442444. 1921775.  210424.  852919.  175925.   15979. 1476079.
  152458.  203044.  270105.   93938.  227869.  142655.  114186.  282749.
   65628.   51429. 2009922.   81208.  896523.   53468. 2781388.  227237.
  131312.  214485.  136352.   32819.  900727.  263719. 1823798. 1260515.
 1881153. 1523354.  699751.]
inertia:  223297.00484976405
